In [88]:
from niio import loaded
from fragmenter import RegionExtractor as re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [81]:
data_dir = '/Users/kristianeschenburg/Desktop/Data/'

In [126]:
# load surface file
surface_file = '%sSurfaces/285345.L.inflated.32k_fs_LR.surf.gii' % (data_dir)
[verts, faces] = loaded.loadSurf(surface_file)

# load cortical map and extract indices of precentral and postcentral gyri
label_file = '%sLabels/Desikan/285345.L.aparc.32k_fs_LR.label.gii' % (data_dir)
R = re.Extractor(label_file)
index_map = R.map_regions()

regions = list(index_map.keys())
regions.sort()

inds = R.indices(index_map, regions=['inferiorparietal'])

In [127]:
conn_file = '%sConnectopy/Desikan/285345/285345.L.inferiorparietal.2.brain.Evecs.func.gii' % (data_dir)
conn = loaded.load(conn_file)
reg_file = '%sCorticalRegionalization/Desikan/RestingState/285345.L.Regionalized.RestingState.aparc.Full.mat' % (data_dir)
reg = loaded.load(reg_file)

In [129]:
conn = conn[inds, 1]

temp_reg = reg[inds, :]
temp_reg = sm.add_constant(temp_reg, prepend=True)

data = pd.DataFrame(temp_reg)
data.columns = ['intercept'] + regions
d1 = data.drop(['corpuscallosum'], axis=1)

In [132]:
M = sm.OLS(endog=conn, exog=data, hasconst=True).fit()

In [133]:
M.rsquared

0.5358917091533294

In [142]:
M.fvalue

60.51826852034556

In [143]:
M.f_pvalue

3.7424259429978023e-268